<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/computer_vision_data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from torch.utils.data.dataset import random_split

from torchvision import transforms
from torchvision.models import resnet50

# 커스텀 데이터셋 만들기

In [2]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

# 이미지 augmentation(증가)

In [3]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [9]:
!unzip -qq /content/drive/My\ Drive/Colab\ Notebooks/computer_vision/2차\ 배포/dirty_mnist_2nd.zip
!unzip -qq /content/drive/My\ Drive/Colab\ Notebooks/computer_vision/2차\ 배포/mnist_data.zip
!unzip -qq /content/drive/My\ Drive/Colab\ Notebooks/computer_vision/2차\ 배포/test_dirty_mnist_2nd.zip

trainset = MnistDataset('/content', '/content/drive/MyDrive/Colab Notebooks/computer_vision/2차 배포/dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('/content', '/content/drive/MyDrive/Colab Notebooks/computer_vision/2차 배포/sample_submission.csv', transforms_test)


# train_loader = DataLoader(trainset, batch_size=256, num_workers=8)
train_loader = DataLoader(trainset,batch_size=32, num_workers=2)

# test_loader = DataLoader(testset, batch_size=32, num_workers=4)
test_loader = DataLoader(testset, batch_size=10, num_workers=1)

# ResNet50 모형

In [10]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
# print(summary(model, (3, 256, 256)))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
# 학습하기

In [13]:
import numpy as np

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.65284, 0.57212
0: 0.65338, 0.59375
0: 0.64205, 0.57692
1: 0.64359, 0.60216
1: 0.64231, 0.59856
1: 0.61096, 0.60337
2: 0.61649, 0.62260
2: 0.61953, 0.60817
2: 0.59024, 0.63702
3: 0.59162, 0.64423
3: 0.59530, 0.65264
3: 0.57262, 0.67188
4: 0.57385, 0.67308
4: 0.58320, 0.66226
4: 0.55857, 0.68269
5: 0.53336, 0.70913
5: 0.53635, 0.70913
5: 0.53931, 0.70192
6: 0.50784, 0.72837
6: 0.52558, 0.72476
6: 0.52206, 0.70673
7: 0.47037, 0.76683
7: 0.48348, 0.75601
7: 0.49109, 0.74159
8: 0.46813, 0.76803
8: 0.46560, 0.76803
8: 0.47236, 0.75962
9: 0.43811, 0.78365
9: 0.43777, 0.77885
9: 0.45023, 0.77163


In [14]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/computer_vision/2차 배포/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()

submit.to_csv('/content/drive/MyDrive/Colab Notebooks/computer_vision/submission_2-1.csv', index=False)